In [4]:
%pip install pandas



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('online_retail_II.csv')

In [8]:
print(f"Data Loaded Successfully! Rows: {df.shape[0]}")
df.head()

Data Loaded Successfully! Rows: 1067371


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [11]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [12]:
df = df.dropna(subset=['Customer ID'])

In [13]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [14]:
df.shape

(824364, 8)

In [15]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [16]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 824364 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      824364 non-null  object        
 1   StockCode    824364 non-null  object        
 2   Description  824364 non-null  object        
 3   Quantity     824364 non-null  int64         
 4   InvoiceDate  824364 non-null  datetime64[ns]
 5   Price        824364 non-null  float64       
 6   Customer ID  824364 non-null  int64         
 7   Country      824364 non-null  object        
 8   TotalPrice   824364 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 62.9+ MB


In [19]:
df['Customer ID'] = df['Customer ID'].astype(int)

In [21]:
import datetime as dt

In [22]:
latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

In [26]:
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate':lambda x :(latest_date - x.max()).days,
    'Invoice':'nunique',
    'TotalPrice':'sum'
})

In [27]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

In [33]:
rfm.head(10)

,Recency,Frequency,Monetary
Customer ID,,,
12346,326,17,-64.68
12347,2,8,5633.32
12348,75,5,2019.40
12349,19,5,4404.54
12350,310,1,334.40
12351,375,1,300.93
12352,36,13,1889.21
12353,204,2,406.76
12354,232,1,1079.40


In [36]:
rfm = rfm[rfm['Monetary'] > 0].copy()

In [37]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

In [38]:
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

In [41]:
rfm.head(10)

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
Customer ID,,,,,,,
12347,2,8,5633.32,4,3,4,434
12348,75,5,2019.40,3,3,3,333
12349,19,5,4404.54,4,3,4,434
12350,310,1,334.40,2,1,1,211
12351,375,1,300.93,2,1,1,211
12352,36,13,1889.21,3,4,3,343
12353,204,2,406.76,2,1,2,212
12354,232,1,1079.40,2,1,3,213
12355,214,2,947.61,2,1,3,213


In [45]:
def segment_customer(df):
    if df['RFM_Score'] == '444':
        return 'Champions'
    elif df['F_Score'] == 4:
        return 'Loyal Customers'
    elif df['M_Score'] == 4:
        return 'Big Spenders'
    elif df['RFM_Score'] == '111':
        return 'Lost Customers'
    elif df['R_Score'] == 1:
        return 'At Risk'
    else:
        return 'Potential'

In [46]:
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

In [47]:
rfm.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
Customer ID,,,,,,,,
12347,2,8,5633.32,4,3,4,434,Big Spenders
12348,75,5,2019.40,3,3,3,333,Potential
12349,19,5,4404.54,4,3,4,434,Big Spenders
12350,310,1,334.40,2,1,1,211,Potential
12351,375,1,300.93,2,1,1,211,Potential


In [48]:
rfm.to_csv('rfm_analysis_final.csv')

In [50]:
rfm[['RFM_Score', 'Segment']].head(100)

,RFM_Score,Segment
Customer ID,,
12347,434,Big Spenders
12348,333,Potential
12349,434,Big Spenders
12350,211,Potential
12351,211,Potential
...,...,...
12443,112,At Risk
12444,434,Big Spenders
12445,422,Potential
